### Нарушение согласования

Предыдущая попытка в задаче согласования рода - предсказывать род и сравнивать с ответом pymorphy

Недостаток - большое количество ошибочных предсказаний в словах, таких как "морской". 
Морской порт и цвет морской волны. Род различается, написание - нет.

Попробуем другой подход - предсказание окончаний.

Начнем с поиска ошибки согласования прилагательных

3 типа:
* новЫЙ
* хорошИЙ
* большОЙ

### 1. Построим бандл, сбалансированный по длине предложения и по корпусу.

In [1]:
from tg.grammar_ru.common import Loc
from tg.grammar_ru.corpus import CorpusReader, CorpusBuilder, BucketCorpusBalancer
from tg.grammar_ru.corpus.corpus_reader import read_data

from yo_fluq_ds import Queryable, Query, fluq

from typing import List, Union
from pathlib import Path

import math
import pandas as pd

In [2]:
CORPUS_NAMES = [
    "books.base.zip", 
    "pub.base.zip"
]
#TODO: add smth else?

CORPUS_LIST = [Loc.corpus_path/corpus_name for corpus_name in CORPUS_NAMES]

In [3]:
CORPUS_LIST

[PosixPath('/home/serhio/Data/1Education/grammar-spring/s2/grammar_ru/data-cache/corpus/books.base.zip'),
 PosixPath('/home/serhio/Data/1Education/grammar-spring/s2/grammar_ru/data-cache/corpus/pub.base.zip')]

In [4]:
def get_sentences_count(corpus_path):
    return sum([
        len(frame.groupby("sentence_id")) 
        for frame in read_data(corpus_path)
    ])

In [5]:
for i, corpus_name in enumerate(CORPUS_NAMES):
    print(f"{corpus_name} = {get_sentences_count(CORPUS_LIST[i])} sentences")

/tmp/ipykernel_61267/3628676302.py:4: DeprecationWarning: Call to deprecated function (or staticmethod) read_data. (Use CorpusReader.read_frames_from_several_corpora)
  for frame in read_data(corpus_path)
  9%|▉         | 231/2533 [00:02<00:26, 86.95it/s] 


KeyboardInterrupt: 

In [5]:
LOG_BASE = math.e
BUCKET_PATH = Loc.corpus_path/"prepare/buckets/buckets.parquet"

In [8]:
BucketCorpusBalancer.build_buckets_frame(CORPUS_LIST, BUCKET_PATH, LOG_BASE)

100%|██████████| 69416/69416 [39:07<00:00, 29.57it/s]  


In [9]:
pd.read_parquet(BUCKET_PATH)

,sentences,bucket_size
bucket,,
books.base.zip/0,"[2105037, 2635119, 2635122, 2930082, 2930089, ...",129
books.base.zip/1,"[7, 8, 11, 15, 10711, 10730, 10731, 10745, 107...",90591
books.base.zip/2,"[1, 2, 6, 9, 12, 14, 16, 17, 18, 19, 20, 21, 2...",434207
books.base.zip/3,"[0, 3, 4, 5, 10, 13, 10712, 10716, 10718, 1072...",355499
books.base.zip/4,"[10715, 10717, 10724, 10744, 10777, 10793, 107...",23177
books.base.zip/5,"[189579, 784022, 2391783, 5783820, 5998276, 67...",65
pub.base.zip/0,"[51423, 71661, 461963, 1207024, 1404782, 25887...",18308
pub.base.zip/1,"[24, 33, 34, 40, 52, 53, 64, 11030, 31091, 311...",60391
pub.base.zip/2,"[3, 5, 6, 8, 10, 14, 18, 21, 25, 26, 28, 30, 3...",415067


In [6]:
BUCKET_NUMBERS = [1, 2, 3, 4]
BUCKET_LIMIT = 60_000 #Note: забили на то что в books.base.zip/4 всего 23К предложений.
# некторый дисбаланс все-же остался

In [11]:
BucketCorpusBalancer.filter_buckets_by_bucket_numbers(BUCKET_PATH, BUCKET_NUMBERS)

In [12]:
pd.read_parquet(BUCKET_PATH)

,sentences,bucket_size
books.base.zip/1,"[7, 8, 11, 15, 10711, 10730, 10731, 10745, 107...",90591
books.base.zip/2,"[1, 2, 6, 9, 12, 14, 16, 17, 18, 19, 20, 21, 2...",434207
books.base.zip/3,"[0, 3, 4, 5, 10, 13, 10712, 10716, 10718, 1072...",355499
books.base.zip/4,"[10715, 10717, 10724, 10744, 10777, 10793, 107...",23177
pub.base.zip/1,"[24, 33, 34, 40, 52, 53, 64, 11030, 31091, 311...",60391
pub.base.zip/2,"[3, 5, 6, 8, 10, 14, 18, 21, 25, 26, 28, 30, 3...",415067
pub.base.zip/3,"[0, 1, 2, 4, 7, 9, 11, 13, 15, 16, 17, 19, 20,...",820561
pub.base.zip/4,"[12, 31092, 132279, 132283, 132285, 132289, 13...",156624


In [7]:
BALANCED_PATH = Loc.corpus_path/"prepare/balanced/books&pub_60K_balanced.zip"

In [8]:
from tg.common import Logger
def balancing() -> None:
    balancer = BucketCorpusBalancer(
        buckets = pd.read_parquet(BUCKET_PATH), 
        log_base = LOG_BASE,
        bucket_limit = BUCKET_LIMIT,
    )

    CorpusBuilder.transfuse_corpus(
        sources = CORPUS_LIST,
        destination = BALANCED_PATH,
        selector = balancer
    )

In [20]:
balancing()

In [19]:
from collections import defaultdict
lengths = defaultdict(int)
for frame in read_data(BALANCED_PATH):
    for corpus_name in CORPUS_NAMES:
        lengths[corpus_name] += len(frame[frame.original_corpus_id == corpus_name].groupby("sentence_id"))

lengths

/tmp/ipykernel_12481/1361944034.py:3: DeprecationWarning: Call to deprecated function (or staticmethod) read_data. (Use CorpusReader.read_frames_from_several_corpora)
  for frame in read_data(BALANCED_PATH):
100%|██████████| 147/147 [00:11<00:00, 12.57it/s]


defaultdict(int, {'books.base.zip': 203177, 'pub.base.zip': 240000})

In [8]:
from tg.grammar_ru import CorpusReader
from tg.grammar_ru.corpus import CorpusBuilder
from pathlib import Path

# corpus = Path('files/corpus.zip')
reader = CorpusReader(BALANCED_PATH)
reader.get_toc()

,filename,timestamp,part_index,token_count,character_count,ordinal,max_id
file_id,,,,,,,
45f6d5b5-3e14-4346-b848-003eed141143,,2023-03-13 20:24:10.177652,0,49971,220227,0,50269
20ddbb6c-2938-454a-8f12-d5114797b9fe,,2023-03-13 20:24:13.361754,1,49976,216755,1,100630
7acf76d8-cfad-4790-a82e-2902263709cd,,2023-03-13 20:24:17.309190,2,49995,208917,2,151076
ab966203-6129-4f22-9dac-fa01e965b601,,2023-03-13 20:24:24.177992,3,49987,209376,3,202229
3670730b-1f7f-4bf1-808b-33717f778f77,,2023-03-13 20:24:29.437513,4,49966,208659,4,253031
...,...,...,...,...,...,...,...
062f6711-5af1-4fa5-8ccf-5bf2d4a95bff,,2023-03-13 21:48:25.985407,142,49992,252939,142,7763628
612d244c-ae4d-4b4c-9076-e9723c63259e,,2023-03-13 21:49:10.772981,143,49984,253894,143,7818826
0bd18961-2d09-4e6c-ba2d-8786f6de8627,,2023-03-13 21:49:58.524944,144,49985,255516,144,7874300


In [9]:
reader.read_frames().first().word.head(20)

0        Собственно
1            потому
2                 ,
3               что
4           перевод
5                от
6            РОСМЭН
7               был
8             очень
9              плох
10                ,
11                в
12        интернете
13        появились
14           другие
15    русскоязычные
16         варианты
17             этой
18           книжки
19                .
Name: word, dtype: object

In [93]:
dfs = reader.read_frames().take(10).to_list()

In [13]:
#NOTE: we need nltk in requirements

In [88]:
import numpy as np
import pandas as pd

import re

from tg.common import DataBundle
from tg.common.ml.batched_training import train_display_test_split
from tg.grammar_ru.features import PyMorphyFeaturizer

from tg.grammar_ru.corpus import ITransfuseSelector
from nltk.stem import SnowballStemmer
from pymystem3 import Mystem
mystem = Mystem()

new = {'ая', 'ого', 'ое', 'ой', 'ом', 'ому',
       'ою', 'ую', 'ые', 'ый', 'ым', 'ыми', 'ых'}

good = {'ая', 'его', 'ее', 'ей', 'ем', 'ему',
        'ие', 'ий', 'им', 'ими', 'их', 'ую', 'яя', 'юю'}

big = {'ая', 'ие', 'им', 'ими', 'их', 'ого',
       'ое', 'ой', 'ом', 'ому', 'ою', 'ую'}

POSSIBLE_ENDINGS = set().union(new, good, big)


def _get_poses_by_sentence(sentence: str):
    # NOTE: краткие прилагательные тоже отмечаются как прилагательные. e.g. Хороша, плох.
    res = []
    for word_info in mystem.analyze(sentence):
        if 'analysis' not in word_info or not word_info["analysis"]:
            continue
        res.append(
            (word_info["text"],
             re.split(
                ',|=', word_info["analysis"][0]["gr"])[0])
        )
    return res


def _set_mystem_pos(df):
    df['pos_mystem'] = np.nan
    for sent_id, tokens_group in df.groupby("sentence_id"):
        sentence = ' '.join(tokens_group.word)
        poses = _get_poses_by_sentence(sentence)
        if not poses:
            continue
        j = 0
        for i in tokens_group.index:  # zip with gaps
            word, pos = poses[j]
            if df.at[i, 'word'] == word:
                df.at[i, 'pos_mystem'] = pos
                j += 1
                if j == len(poses):
                    break


def _extract_true_ending(stemmed_ending: str):
    for possible_ending in POSSIBLE_ENDINGS:  # TODO can we make it faster?
        if stemmed_ending.lower().endswith(possible_ending):
            return possible_ending
    return np.nan


class AdjAgreementTrainIndexBuilder(ITransfuseSelector):
    def __init__(self):
        self.pmf = PyMorphyFeaturizer()
        self.snowball = SnowballStemmer(language="russian")

    def select(self, source, df, toc_row):  # ~build_train_index
        def get_ending(row):
            return row.word[row.stem_len:]

        def get_norm_form_ending(row):
            # name - индекс строки.
            stem_len = adjectives.at[row.name, 'stem_len']
            return row.normal_form[stem_len:]

        df['label'] = -1
        _set_mystem_pos(df)
        db = DataBundle(src=df)
        self.pmf.featurize(db)  # запишет результат по ключу pymorphy
        morphed = db.data_frames['pymorphy']
        morphed.replace({np.nan: 'nan'}, inplace=True)
        df['is_target'] = df.pos_mystem == 'A'
        adjectives = df[
            (df.pos_mystem == 'A') &
            # (~morphed.POS.isin(["ADJS", "ADVB"]))
             (morphed.POS == "ADJF")
        ].copy()  # TODO delete

        # adjectives['stem_len'] = adjectives.word.apply(
        #     self.snowball.stem).apply(len)
        # adjectives['ending'] = adjectives.apply(get_ending, axis=1)
        adjectives['true_ending'] = (adjectives.word
                                     .apply(_extract_true_ending))

        morphed_adjectives = morphed.loc[adjectives.index]
        # adjectives['stem_len'] = morphed_adjectives.normal_form.apply(
        #     self.snowball.stem).apply(len)
        # adjectives['norm_ending'] = (morphed_adjectives
        #                              .apply(get_norm_form_ending, axis=1))
        adjectives['true_norm_ending'] = (morphed_adjectives.normal_form
                                          .apply(_extract_true_ending))

        adjectives['norm_form'] = morphed_adjectives.normal_form
        return adjectives
        gender_scores_cols = [
            'gender_masc_score', 'gender_femn_score', 'gender_neut_score', 'gender_None_score']
        # 'gender_masc_score', 'gender_femn_score', 'gender_neut_score', 'gender_None_score'
        df[gender_scores_cols] = morphed[gender_scores_cols]
        df.loc[df.is_target, 'label'] = morphed[df.is_target].gender.replace(
            self.gender_nums)
        return [df]

    @staticmethod
    def build_index_from_src(src_df):
        df = src_df.loc[src_df.is_target][[
            'word_id', 'sentence_id', 'label']].copy()
        df = df.reset_index(drop=True)
        df.index.name = 'sample_id'
        df['split'] = train_display_test_split(df)
        return df


In [96]:
index_builder = AdjAgreementTrainIndexBuilder()
dfs = reader.read_frames().take(10).to_list()

adjs = index_builder.select(_, pd.concat(dfs), _)


In [101]:
# set(adjs
#  [adjs.true_norm_ending.isnull() | adjs.true_ending.isnull()]
#  [['word',  'true_ending', 'norm_form', 'true_norm_ending']].word)


окончание не определено:
{'Адова',
 'Ведьмином',
 'Волчьи',
 'Выдающихся',
 'Дядиного',
 'Кабанью',
 'Летучемышиному',
 'Мамина',
 'Манфри',
 'Отцовы',
 'Разбойничья',
 'Светящийся',
 'Старухин',
 'Старухиным',
 'бараньи',
 'беличье',
 'божье',
 'верблюжьи',
 'волчье',
 'волчьи',
 'волчью',
 'волчья',
 'выдающегося',
 'выдающиеся',
 'вьющейся',
 'вьющиеся',
 'вьющимися',
 'вьющимся',
 'вьющихся',
 'говяжьи',
 'драконову',
 'драконье',
 'драконьи',
 'драконья',
 'кабанье',
 'кабаньи',
 'кажущейся',
 'кажущимся',
 'кроличьи',
 'маминого',
 'медвежью',
 'мужнино',
 'невольничье',
 'негнущиеся',
 'непрекращающийся',
 'несостоявшимся',
 'неудавшимся',
 'неулыбающееся',
 'овечьи',
 'оленьи',
 'оленья',
 'поросячьи',
 'поросячью',
 'праздношатающиеся',
 'рыбачьи',
 'рыбью',
 'светящейся',
 'светящиеся',
 'светящимся',
 'светящихся',
 'светящуюся',
 'сестрину',
 'скоморошьи',
 'сорочьи',
 'стелющиеся',
 'телячьи',
 'тюленьи',
 'чертову',
 'ягнячьи'}

In [78]:
(adjs[adjs.true_norm_ending.isnull()]
 [['word', 'ending', 'true_ending', 'norm_form', 'norm_ending', 'true_norm_ending']])


,word,ending,true_ending,norm_form,norm_ending,true_norm_ending
24217,Летучемышиному,ому,ому,летучемышин,,NaN


In [25]:
adjs.ending.unique()

array(['ые', 'ого', 'ый', 'ых', 'ой', 'ое', 'их', 'ей', 'ую', 'ий', 'ему',
       'ном', 'ом', 'ем', 'ной', 'его', 'ного', 'ая', 'яя', 'нная', 'ому',
       'ейшей', 'ный', 'ующие', 'ым', 'нным', 'ных', 'ие', 'ими', 'ыми',
       'ная', 'ному', 'ное', 'ными', 'им', 'ьих', 'ную', 'ные', 'ующий',
       'нные', 'нных', 'щей', 'щих', 'ья', 'нное', 'ным', 'нном', 'ОГО',
       'нного', 'них', 'нными', 'юю', 'щий', 'ейшее', 'щая', 'ующим',
       'ующее', 'ее', 'ний', 'ней', 'щим', 'щего', 'нный', 'нной', 'ющих',
       'ующих', 'ьи', 'ейшем', 'нную', 'ющей', 'ейшая', 'щие', 'емым',
       'ье', 'него', 'ейшему', 'ейшего', 'ующую', 'емых', 'й', 'ейшие',
       'ью', 'щее', 'щими', 'нейшие', 'нем', 'ующими'], dtype=object)

новый 'ый ого ому ым ом ое ая ой ую ою ые ых ыми'
хороший ['ий его ему им ем ее его ему ее им ем ая ей ую ей ую ие их им ими ']
большой ['ой ого ому им ом ое ого ому ое им ом ая ой ую ой ою ие их им ими']

In [34]:
# set('ий его ему им ем ее его ему ее им ем ая ей ую ей ую ие их им ими'.split())
# set('ый ого ому ым ом ое ая ой ую ою ые ых ыми'.split())
# set('ой ого ому им ом ое ого ому ое им ом ая ой ую ой ою ие их им ими'.split())

In [16]:
adjs.norm_ending.value_counts()

ый       1159
ий        439
ный       163
ой        159
нный       33
ующий      24
щий        20
ний         6
ейший       5
ющий        2
емый        2
            1
й           1
Name: norm_ending, dtype: int64

In [23]:
# adjs[adjs.norm_ending=='ующий']

In [54]:
for norm_end, group in adjs.groupby('norm_ending'):
    print(group[['word', 'ending', 'true_ending', 'norm_form', 'norm_ending', 'true_norm_ending']])
    print('####')

                 word ending true_ending    norm_form norm_ending  \
24217  Летучемышиному    ому         ому  летучемышин               

      true_norm_ending  
24217             None  
####
             word ending true_ending   norm_form norm_ending true_norm_ending
11202  дальнейшее  ейшее          ее  дальнейший       ейший               ий
23089  дальнейшем  ейшем          ем  дальнейший       ейший               ий
31198  дальнейшее  ейшее          ее  дальнейший       ейший               ий
35694  дальнейшие  ейшие          ие  дальнейший       ейший               ий
36954  дальнейшее  ейшее          ее  дальнейший       ейший               ий
####
                word ending true_ending      norm_form norm_ending  \
26548  непроницаемым   емым          ым  непроницаемый        емый   
33346   нескончаемых   емых          ых   нескончаемый        емый   

      true_norm_ending  
26548               ый  
33346               ый  
####
               word ending true_ending    

In [17]:
adjs.ending.value_counts()


ой        240
ый        188
ые        146
ого       138
ым        131
         ... 
ней         1
нными       1
них         1
ОГО         1
ующими      1
Name: ending, Length: 87, dtype: int64

,word,ending,norm_form,norm_ending
15,русскоязычные,ые,русскоязычный,ый
20,Приятного,ого,приятный,ый
96,злосчастный,ый,злосчастный,ый
104,неприятных,ых,неприятный,ый
108,долгой,ой,долгий,ий
...,...,...,...,...
50101,хогвартском,ом,хогвартский,ий
50145,грубым,ым,грубый,ый
50162,чужие,ие,чужой,ой
50183,двойной,ой,двойной,ой


In [75]:
snowball = SnowballStemmer(language="russian")
snowball.stem("хорошего")

'хорош'

In [136]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [142]:
morph.parse('виновато')

[Parse(word='виновато', tag=OpencorporaTag('ADVB'), normal_form='виновато', score=0.5, methods_stack=((DictionaryAnalyzer(), 'виновато', 3, 0),)),
 Parse(word='виновато', tag=OpencorporaTag('ADJS,Qual neut,sing'), normal_form='виноватый', score=0.5, methods_stack=((DictionaryAnalyzer(), 'виновато', 249, 56),))]

In [22]:
from pymystem3 import Mystem
m = Mystem()

In [23]:

m.lemmatize("Красивая мама красиво мыла раму")

['красивый', ' ', 'мама', ' ', 'красиво', ' ', 'мыть', ' ', 'рама', '\n']

In [24]:
text = 'Красивая мама красиво мыла раму'
import json
json.dumps(m.analyze(text), ensure_ascii=False)

'[{"analysis": [{"lex": "красивый", "wt": 1, "gr": "A=им,ед,полн,жен"}], "text": "Красивая"}, {"text": " "}, {"analysis": [{"lex": "мама", "wt": 1, "gr": "S,жен,од=им,ед"}], "text": "мама"}, {"text": " "}, {"analysis": [{"lex": "красиво", "wt": 0.8149252476, "gr": "ADV="}], "text": "красиво"}, {"text": " "}, {"analysis": [{"lex": "мыть", "wt": 0.441520999, "gr": "V,несов,пе=прош,ед,изъяв,жен"}], "text": "мыла"}, {"text": " "}, {"analysis": [{"lex": "рама", "wt": 0.9993591156, "gr": "S,жен,неод=вин,ед"}], "text": "раму"}, {"text": "\\n"}]'

In [25]:
type(m.analyze('мама сделала так, как сказал ваш начальник')[0])

dict

In [26]:
import re
#мама сделала деревянную шкатулку так, как сказал первый начальник

In [27]:
sentences.iloc[4]

'Диффиндо — заклятие ножниц , разрезает ткань .'

In [28]:
df[df.sentence_id ==4]

,word_id,sentence_id,word_index,paragraph_id,word_tail,word,word_type,word_length,file_id,corpus_id,original_corpus_id,original_word_id,original_sentence_id,original_paragraph_id,updated,label
29,29,4,0,4,1,Диффиндо,ru,8,45f6d5b5-3e14-4346-b848-003eed141143,books&pub_60K_balanced.zip,books.base.zip,304,34,26,False,-1
30,30,4,1,4,1,—,punct,1,45f6d5b5-3e14-4346-b848-003eed141143,books&pub_60K_balanced.zip,books.base.zip,305,34,26,False,-1
31,31,4,2,4,1,заклятие,ru,8,45f6d5b5-3e14-4346-b848-003eed141143,books&pub_60K_balanced.zip,books.base.zip,306,34,26,False,-1
32,32,4,3,4,0,ножниц,ru,6,45f6d5b5-3e14-4346-b848-003eed141143,books&pub_60K_balanced.zip,books.base.zip,307,34,26,False,-1
33,33,4,4,4,1,",",punct,1,45f6d5b5-3e14-4346-b848-003eed141143,books&pub_60K_balanced.zip,books.base.zip,308,34,26,False,-1
34,34,4,5,4,1,разрезает,ru,9,45f6d5b5-3e14-4346-b848-003eed141143,books&pub_60K_balanced.zip,books.base.zip,309,34,26,False,-1
35,35,4,6,4,0,ткань,ru,5,45f6d5b5-3e14-4346-b848-003eed141143,books&pub_60K_balanced.zip,books.base.zip,310,34,26,False,-1
36,36,4,7,4,0,.,punct,1,45f6d5b5-3e14-4346-b848-003eed141143,books&pub_60K_balanced.zip,books.base.zip,311,34,26,False,-1


In [113]:
a = """Он пошевелил больной нога $400. Больной встала с кровати.
Цвет морской волны. Морской порт. competition is - awesome"""

for word_info in mystem.analyze('один, тот, этот, сей, ваш'):
    if 'analysis' not in word_info or not word_info["analysis"]:
        continue
    # print(word_info)
    print(word_info['text'], re.split(
        ',|=', word_info["analysis"][0]["gr"])[0])


один APRO
тот APRO
этот APRO
сей APRO
ваш APRO


In [152]:
get_poses_by_sentence(sentences.iloc[0])

[('Собственно', 'ADV'),
 ('потому', 'ADVPRO'),
 ('что', 'CONJ'),
 ('перевод', 'S'),
 ('от', 'PR'),
 ('РОСМЭН', 'S'),
 ('был', 'V'),
 ('очень', 'ADV'),
 ('плох', 'A'),
 ('в', 'PR'),
 ('интернете', 'S'),
 ('появились', 'V'),
 ('другие', 'APRO'),
 ('русскоязычные', 'A'),
 ('варианты', 'S'),
 ('этой', 'APRO'),
 ('книжки', 'S')]

In [46]:
len(df)

49971

In [47]:
df.at[0, 'word']

'Собственно'

In [48]:
df.index

Int64Index([    0,     1,     2,     3,     4,     5,     6,     7,     8,
                9,
            ...
            50258, 50259, 50260, 50261, 50262, 50263, 50264, 50265, 50266,
            50267],
           dtype='int64', length=49971)

In [49]:
get_pos(df)

[('Собственно', 'ADV'), ('потому', 'ADVPRO'), ('что', 'CONJ'), ('перевод', 'S'), ('от', 'PR'), ('РОСМЭН', 'S'), ('был', 'V'), ('очень', 'ADV'), ('плох', 'A'), ('в', 'PR'), ('интернете', 'S'), ('появились', 'V'), ('другие', 'APRO'), ('русскоязычные', 'A'), ('варианты', 'S'), ('этой', 'APRO'), ('книжки', 'S')]
####
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
[('Приятного', 'A'), ('чтения', 'S')]
####
20
21
[('Игорь', 'S'), ('Шенин', 'S')]
####
23
24
[('Вот', 'PART'), ('их', 'SPRO'), ('перечень', 'S')]
####
25
26
27
[('Диффиндо', 'S'), ('заклятие', 'S'), ('ножниц', 'S'), ('разрезает', 'V'), ('ткань', 'S')]
####
29
30
31
32
33
34
35
[('Инсендио', 'S'), ('экспеллимеллиус', 'S'), ('поджигает', 'V'), ('объект', 'S')]
####
37
38
39
40
41
42
[('Орхидеус', 'S'), ('создаёт', 'V'), ('букет', 'S'), ('орхидей', 'S')]
####
44
45
46
47
48
[('Экспульсо', 'S'), ('конфринго', 'S'), ('взрывает', 'V'), ('объект', 'S')]
####
50
51
52
53
54
55
[('Эпискей', 'S'), ('прекращает', 'V'), ('кровотечение', 'S'),

In [58]:
df[["word","pos_mystem"]][400:450]

,word,pos_mystem
411,политиков,S
412,",",NaN
413,и,CONJ
414,обычно,ADV
415,это,SPRO
416,не,PART
417,предвещало,V
418,ничего,SPRO
419,хорошего,A
420,.,NaN


In [92]:
m.analyze('больной ногой')

[{'analysis': [{'lex': 'больной',
    'wt': 0.5204771579,
    'gr': 'A=(вин,ед,полн,муж,неод|им,ед,полн,муж|пр,ед,полн,жен|дат,ед,полн,жен|род,ед,полн,жен|твор,ед,полн,жен)'}],
  'text': 'больной'},
 {'text': ' '},
 {'analysis': [{'lex': 'нога', 'wt': 1, 'gr': 'S,жен,неод=твор,ед'}],
  'text': 'ногой'},
 {'text': '\n'}]

In [98]:
m.analyze('больной')


[{'analysis': [{'lex': 'больной', 'wt': 0.4129127344, 'gr': 'S,муж,од=им,ед'}],
  'text': 'больной'},
 {'text': '\n'}]